# Using `numba.jit` to speedup the computation of the Cityblock distance matrix 


In this notebook we implement a function to compute the Cityblock distance matrix using Numba's *just-it-time* compilation decorator. We compare it's performance to that of corresponding non-decorated NumPy function.

We will use two Numba functions here. The decorator ` @numba.jit` and `numba.prange`.

In [4]:
import numpy as np
import numba

In [5]:
def cityblock_python(x, y):
    """Cityblock distance matrix."""

    num_samples, num_feat = x.shape
    dist_matrix = np.empty((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in range(num_feat):
                r += np.abs(x[i][k] - y[j][k])
            dist_matrix[i][j] = r
    return dist_matrix


@numba.jit(nopython=True)
def cityblock_numba1(x, y):
    """Cityblock distance matrix."""

    num_samples, num_feat = x.shape
    dist_matrix = np.empty((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in numba.prange(num_feat):
                r += np.abs(x[i][k] - y[j][k])
            dist_matrix[i][j] = r

    return dist_matrix


@numba.jit(nopython=True)
def cityblock_numba2(x, y):
    """Cityblock distance matrix using `numpy.linalg.norm`
    operation.
    """
    num_samples, num_feat = x.shape
    dist_matrix = np.empty((num_samples, num_samples))
    for i in range(num_samples):
        for j in numba.prange(num_samples):
            dist_matrix[i][j] = np.linalg.norm(x[i] - y[j], 1)

    return dist_matrix

### Note
Observe that we do the inner loop, which is a reduction, with `numba.prange`. `numba.prange` automatically takes care of data privatization and reductions.

In [6]:
# Let's check that they all give the same result
a = 10. * np.random.random([100, 10])

print(np.abs(cityblock_python(a, a) - cityblock_numba1(a, a)).max())
print(np.abs(cityblock_python(a, a) - cityblock_numba2(a, a)).max())

0.0
0.0


In [7]:
nsamples = 200
nfeat = 25

x = 10. * np.random.random([nsamples, nfeat])

%timeit cityblock_python(x,x)
%timeit cityblock_numba1(x, x)
%timeit cityblock_numba2(x, x)

1.8 s ± 23.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
777 µs ± 2.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
6.39 ms ± 61.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Exercise 1
How do you explain the difference in execution times?

## Conclusions

In cases where there's no possibility to do an implementation with NumPy vectorized operations, it's worth to give a try to Numba. It offers a significant improvement in performance compared to pure python, specially in situations where loops are unavoidable.

As we have seen, the speedup doesn't come completelly for free: the way the python function is implmented is crucial to obtain a good performance from Numba. Consider different implementations with and without NumPy operations and measure their execution time.